In [1]:
import  requests
import json

In [41]:
def json_load(res):
  t = json.dumps(res.json())
  return json.loads(t)

def prepare_api(word:str):
    headers = {
        "Accept": "application/json",
        "app_id": "8beabadc",
        "app_key": "4d85f3e2e7cd293da9a811f156c99841"
    }
    word_id = word
    url_entries = 'https://od-api.oxforddictionaries.com/api/v2/entries/en/' + word_id + '?strictMatch=true'
    res_entries = requests.get(url_entries, headers = {'app_id' : headers['app_id'], 'app_key' : headers['app_key']})
    url_sentences = 'https://od-api.oxforddictionaries.com/api/v2/sentences/en/' + word_id + '?strictMatch=true'
    res_sentences = requests.get(url_sentences, headers = {'app_id' : headers['app_id'], 'app_key' : headers['app_key']})

    senses_examples = json_load(res_entries)
    sentences_examples = json_load(res_sentences)

    return senses_examples, sentences_examples

def run_words(word:str):

    senses_examples, sentences_examples = prepare_api(word)

    api_call_senses = senses_examples['results'][0]['lexicalEntries'][0]['entries'][0]['senses']
    senses = []
    sense_with_examples = {}
    diff_sense_ids = []
    sentences = sentences_examples['results'][0]['lexicalEntries'][0]['sentences']

    for el in sentences:
        diff_sense_ids.append(el['senseIds'][0])

    sense_ids = set(diff_sense_ids)

    def search(id):
        return list(sent['text'] for sent in sentences if sent['senseIds'][0] == id)

    for idx, sens in enumerate(api_call_senses):

        sense_with_examples['sense'] = idx
        sense_with_examples['definition'] = sens['definitions'][0]

        if sens['id'] in list(sense_ids):
            examples_sense = search(sens['id'])
            sense_with_examples['examples'] = examples_sense

        senses.append(sense_with_examples.copy())

    return senses

In [43]:
run_words('meeting')

[{'sense': 0,
  'definition': 'an assembly of people for a particular purpose, especially for formal discussion',
  'examples': ['They also became focal points for group discussion and formal meetings.',
   'Union officials have written to executives seeking an urgent meeting to discuss the matter.',
   'The purpose of the meeting was to discuss the merits of the former ambassador traveling to Niger.',
   'The purpose of the meeting is to discuss the set-up of a local Farmers Market planned for the near future.',
   'It was the first time in 15 years they had held a formal meeting to discuss problems in the area.',
   'They understood the purpose of the meeting was to discuss incorporating a new company.',
   'Health bosses held urgent meetings to decide how to cope with an expected surge in the number of patients over Easter.',
   'The actual discussion at meetings is not recorded formally, or contained in the minutes.',
   "The main item for discussion at the meeting was the guild's 